In [35]:
import pandas as pd
import json
import numpy as np

In [3]:
df = pd.read_csv("data/www2017/wikihowURLID-wikihowURL-youtubeURL", sep="\t", names=["fid", "wikihow", "youtube"])
df['query'] = [i.split("/")[-1].replace("-"," ") for i in df.wikihow.tolist()]
df['label'] = [i.split("/")[-1] for i in df.youtube.tolist()]

In [4]:

def read_youtube_video_json():
    vids, titles, descs, tags, comments = [], [], [], [], []
    with open("data/www2017/uniq-youtube-video.json", encoding="utf-8") as file:
        for l in file:
            y = json.loads(l)
            vids.append(y['id'])
            titles.append(y['title'])
            descs.append(y['description'])
            tags.append(' '.join(y['tags']))
            comments.append(' '.join([i['comment'] for i in y['comment']]))
    
    return pd.DataFrame({'vid':vids, 'title':titles, 'desc':descs, 'comment':comments, 'tag':tags})

df_video = read_youtube_video_json()

In [5]:

def get_wikihow_with_video():
    linkids = []
    with open("data/www2017/task-frame-have-video.json", encoding="utf-8") as file:
        for l in file:
            y = json.loads(l)
            linkids.append(int(y['activity']['linkid']))
    return linkids
wikihow_with_video = get_wikihow_with_video()

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df_video.title.tolist()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [24]:
query_vec = vectorizer.transform(df['query'])

In [47]:
query_vec.shape

(18054, 15223)

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate():
    scores = cosine_similarity(query_vec, X)
    idx = 

In [32]:
sorted(x, key=x.get, reverse=True)

In [46]:
scores.shape

(18054, 17608)

In [49]:
x = {'a':10.1, 'b':2, 'c':5}

In [52]:
x.get('a')

10.1

In [ ]:

def getHitRatio(ranklist, gtDoc, K):
    for i in range(K):
        item = ranklist[i]
        if item == gtDoc:
            return 1
    return 0

def getMRR(ranklist, gtDoc):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtDoc:
            return 1.0 / (i+1)
    return 0

hr1s, hr3s, hr5s, hr10s, mrrs = [], [], [], [], []
for score, gtDoc in zip(scores, df.label):
    ranklist = {i:j for i,j in zip(df_video.vid, score)}
    ranklist = sorted(ranklist, key=ranklist.get, reverse=True)
    hr1 = getHitRatio(ranklist, gtDoc, 1)
    hr3 = getHitRatio(ranklist, gtDoc, 3)
    hr5 = getHitRatio(ranklist, gtDoc, 5)
    hr10 = getHitRatio(ranklist, gtDoc, 10)
    mrr = getMRR(ranklist, gtDoc)
    mrrs.append(mrr)
print(np.mean(mrrs))
